# Peformance comparison of different algorithms

In this notebook we analyze the performance of various algorithms, including MLPs, dense dynamical systems, and various sorts of sparse dynamical systems.

In [15]:
import torch
import warnings
import time
warnings.filterwarnings("ignore")
# TODO Add these imports and test MaskeLinear and SparseLinear
from iterativennsimple.MaskedLinear import MaskedLinear
from iterativennsimple.SparseLinear import SparseLinear

In [ ]:
def generate(size, entries, device="cuda"):
    """create a variety of sparse matrices with the same entries

    Args:
        size (int, optional): Size of the square matrix. Defaults to 1000.
        entries (int, optional): Total number of non-zero entries. Note this is an upper bound, but should be close to the actual size. 
        device (str, optional): "cuda" or "cpu". Defaults to "cuda".

    Returns:
        dict: Dictionary of the sparse matrices
    """ 
    output = {}
    # We first create a COO tensor since that is easier to create

    # We create a set of random indices and values
    # Each index is a pair of (row, column) indices that indicate the location of the non-zero entry 
    # whose value is stored in the vals tensor.
    # NOTE: This is memory efficient in that we never need to create a dense matrix
    indices = torch.randint(0, size, (entries,2))
    vals = torch.randn(entries)

    # We then create the COO tensor
    coo = torch.sparse_coo_tensor(indices.t(), vals, (size, size), device=device)

    # We then coalesce it to make sure there are no duplicate indices
    coo = coo.coalesce()

    # Then we convert it to CSC and CSR, which are two common sparse matrix formats
    # NOTE: CSR is what we really want for actual use, since it is the most efficient for matrix-vector multiplication on a GPU
    csc = coo.to_sparse_csc()
    csr = coo.to_sparse_csr()

    # We also create a dense version of the matrix for comparison
    # NOTE: This is not memory efficient, since many entries will be 0 and not really needed, but is useful for comparison
    dense = coo.to_dense()

    # We also create the MaskedLinear and SparseLinear objects

    # NOTE: Matrices and modules are transposed in PyTorch, see for example https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
    # This is why we transpose the input to and output from the module when we define the matmul operator "@" below
    class moduleWrapper(object):
        def __init__(self, module, device=device):
            self.module = module
            self.device = device
        def __matmul__(self, x):
            return self.module(x.T).T
        
    maskedLinear = moduleWrapper(MaskedLinear.from_coo(coo).to(device), device)
    # FIXME: 
    sparseLinear = moduleWrapper(SparseLinear.from_coo(coo).to(device), device)

    output["dense"] = dense
    output["coo"] = coo
    output["csc"] = csc
    output["csr"] = csr
    output["maskedLinear"] = maskedLinear
    output["sparseLinear"] = sparseLinear
    return output

In [17]:
## check that all of the matrices are the same operator
def matrix_check(size, entries, device):
    # Generate the matrices
    matrices = generate(size, entries, device=device)
    # Size of the RHS
    x_cols = 100
    x = torch.randn(size, x_cols, device=device)

    print('Matrix-matrix multiplication check')
    y_true = None
    base_name = None

    for matrix, matrix_name in zip(matrices.values(), matrices.keys()):
        y = matrix @ x
        if y_true is None:
            y_true = y
            base_name = matrix_name
        else:
            # print the frobenius norm of the difference
            print(f"||{matrix_name} - {base_name}||_F: {torch.norm(y-y_true)}")
            

In [18]:
matrix_check(1000, 23*1000, "cuda")

Matrix-matrix multiplication check
||coo - dense||_F: 0.00014716177247464657
||csc - dense||_F: 0.00014716495934408158
||csr - dense||_F: 0.00014716577425133437
||maskedLinear - dense||_F: 0.00013371254317462444
||sparseLinear - dense||_F: 0.00016874454740900546


In [19]:
def run_timing(size, entries, device, syncgpu):
    # test if cuda is available
    if device == "cuda":
        if not torch.cuda.is_available():
            print("CUDA is not available, using CPU instead")
            device = "cpu"

    print(f"Running on {device}")
    print(f"Size: {size}")
    print(f"Entries: {entries}")
    print(f"Synchronize GPU: {syncgpu}")
    
    # Generate the matrices
    matrices = generate(size, entries, device=device)

    # Size of the RHS
    x_cols = 100

    # Compute the timings
    print('Matrix-matrix multiplication timings:')
    base_time = None
    base_name = None
    for matrix, matrix_name in zip(matrices.values(), matrices.keys()):
        # first, do a few runs to warm up the cache
        for i in range(2):
            x = torch.randn(size, x_cols, device=device)
            y = matrix @ x
        # now do the timings
        matrix_times = []
        for i in range(5):
            x = torch.randn(size, x_cols, device=device)    
            if syncgpu:
                torch.cuda.synchronize()
            start = time.perf_counter()
            y = matrix @ x
            if syncgpu:
                torch.cuda.synchronize()
            matrix_time = time.perf_counter()-start
            matrix_times.append(matrix_time)
        avg_matrix_time = sum(matrix_times)/len(matrix_times)
        min_matrix_time = min(matrix_times)
        max_matrix_time = max(matrix_times)
        if base_time is None:
            base_time = avg_matrix_time
            base_name = matrix_name

        print(f"{matrix_name} avg time:", avg_matrix_time)
        print(f"{matrix_name} min time:", min_matrix_time)
        print(f"{matrix_name} max time:", max_matrix_time)
        print(f"{matrix_name} speedup over {base_name}:", base_time/avg_matrix_time)

In [20]:
size = 10000
sparsity = 0.1
run_timing(10000, int(10000*10000*sparsity), "cuda", True)

Running on cuda
Size: 10000
Entries: 10000000
Synchronize GPU: True
Matrix-matrix multiplication timings:
dense avg time: 0.0006243461742997169
dense min time: 0.0006222957745194435
dense max time: 0.0006271353922784328
dense speedup over dense: 1.0
coo avg time: 0.0019150343723595142
coo min time: 0.001913445070385933
coo max time: 0.0019187149591743946
coo speedup over dense: 0.3260234820383197
csc avg time: 0.007681993581354618
csc min time: 0.007674871012568474
csc max time: 0.007691313978284597
csc speedup over dense: 0.08127397760590445
csr avg time: 0.0008966541849076748
csr min time: 0.0008951709605753422
csr max time: 0.0008984971791505814
csr speedup over dense: 0.6963065413719154
maskedLinear avg time: 0.003192117623984814
maskedLinear min time: 0.0031884140335023403
maskedLinear max time: 0.003196349833160639
maskedLinear speedup over dense: 0.19558996498391162
sparseLinear avg time: 0.027831985056400298
sparseLinear min time: 0.02780646225437522
sparseLinear max time: 0.02